# Evaluating a Traceable Function

If you are tracing some application using LangSmith's `@traceable` decorator, the `run_on_dataset` function should
automatically configure it to include all runs within the appropriate trace if it is directly passed in to the function.

This functionality is supported in `langchain>=0.0.324`.

In [2]:
from typing import List

from langchain.schema import Document
from langchain.schema.retriever import BaseRetriever


class MinimalRetriever(BaseRetriever):
    def __init__(self):
        super().__init__()

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Implement your retrieval logic here
        # Return a list of relevant documents based on the query
        return [Document(page_content=query)]

In [18]:
import openai
from langchain.schema.runnable import RunnableLambda
from langsmith import traceable

retriever = MinimalRetriever()


@traceable(run_type="llm")
def complete(*args, **kwargs):
    return openai.ChatCompletion.create(
        *args,
        **kwargs,
    )


@traceable()
def my_model(question: str, context: str):
    messages = [
        {
            "role": "system",
            "content": f"Respond to the user, taking into account the context: {context}",
        },
        {
            "role": "user",
            "content": question,
        },
    ]
    completion = complete(model="gpt-3.5-turbo", messages=messages)
    # It should still work if a LangChain component is called within a traceable function
    format = RunnableLambda(lambda x: x.choices[0].message.content) | retriever
    return format.invoke(completion, config={"callbacks": []})

In [19]:
import uuid

from langsmith import Client

client = Client()
uid = uuid.uuid4()
examples = [
    {
        "question": "What's 2+2",
        "context": "You are a pirate",
    },
    {
        "question": "Where did coffee originate from?",
        "context": "You are a knight of King Arthur.",
    },
]
dataset_name = f"Evaluating Traceables Walkthrough - {uid}"
dataset = client.create_dataset(dataset_name)

client.create_examples(
    inputs=examples,
    dataset_id=dataset.id,
)

In [20]:
from langchain.callbacks.manager import CallbackManager, LangChainTracer
from langchain.schema.runnable import RunnableLambda
from langsmith.run_trees import RunTree


class RunnableTraceable(RunnableLambda):
    def __init__(
        self,
        func,
        afunc=None,
    ) -> None:
        def _configure_run_tree(callback_manager):
            run_tree: Optional[RunTree] = None
            if isinstance(callback_manager, CallbackManager):
                lc_tracers = [
                    handler
                    for handler in callback_manager.handlers
                    if isinstance(handler, LangChainTracer)
                ]
                if lc_tracers:
                    lc_tracer = lc_tracers[0]
                    run_tree = RunTree(
                        id=callback_manager.parent_run_id,
                        session_name=lc_tracer.project_name,
                        name="Wrapping",
                        run_type="chain",
                        inputs={},
                    )
            return run_tree

        def wrap_traceable(inputs: dict, config: dict):
            run_tree = _configure_run_tree(config.get("callbacks"))
            return func(**inputs, langsmith_extra={"run_tree": run_tree})

        async def awrap_traceable(inputs: dict, config: dict):
            run_tree = _configure_run_tree(config.get("callbacks"))
            return await afunc(**inputs, langsmith_extra={"run_tree": run_tree})

        awrapped = None
        if afunc is not None:
            awrapped = awrap_traceable

        super().__init__(wrap_traceable, awrapped)

## Promoting to LangChain component

In general, it is recommended to use LangChain runnables throughout your LLM application wherever you want tracing and callbacks. Beta support for composing `@traceable` functions with other LangChain runnables is provided via the `RunnableTraceable` class.

In [40]:
from langchain.chat_models import ChatOpenAI
from langsmith.run_helpers import as_runnable

chain = (
    as_runnable(my_model) | ChatOpenAI()
)  # the second call will respond to the first

In [41]:
chain.invoke(
    {
        "question": "Why is blue brown?",
        "context": "You are a pirate",
    }
)

ValidationError: 1 validation error for ChatPromptValue
messages -> 0 -> content
  field required (type=value_error.missing)